In [5]:
import sys
sys.path.append('../')
from time import time
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm.session import Session

def conn_engine(username: str, password: str, host: str, db_name: str, **_kwargs):
    db_url = f"mysql+pymysql://{username}:{password}@{host}:3306/{db_name}"
    return create_engine(db_url)

def connect_db(username: str, password: str, host: str, db_name: str, **_kwargs) -> Session:

    engine = conn_engine(username, password, host, db_name)
    session_local = sessionmaker(bind=engine)
    return session_local()

secret = {
        "username": "admin",
        "password": "1q2w3e4r!",
        "host": "dodomoards.ccoalf3s8d7c.ap-northeast-2.rds.amazonaws.com",
        "db_name": "captured",
    }

def execute_query(query):
    """esecute_query"""
    db_session = connect_db(**secret)
    result_list = db_session.execute(query)
    if db_session.is_active:
        db_session.close()
    return list(result_list)

s_1 = time()
x = execute_query("SELECT * FROM product_info ")
s_2 = time()

print("query executing", f"{s_2 - s_1:.5f} sec")


query executing 0.15868 sec


In [4]:
import pandas as pd

df = pd.DataFrame(x)
df['size'] = df['size'].apply(lambda x: eval(x))

In [13]:
df = df.explode('size')

df = df[['sku','size']]

df = df.drop_duplicates()

In [23]:
from datetime import datetime
df['available'] = True
df['updated_at'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [26]:
df.to_sql('size', conn_engine(**secret), if_exists='append', index=False)

228

In [50]:
from tables import ProductInfoTable, SizeTable
from sqlalchemy import func,case
session = connect_db(**secret)

In [106]:
result = (
    session.query(ProductInfoTable)
    .join(SizeTable, ProductInfoTable.sku == SizeTable.sku)
    .filter(ProductInfoTable.sku < 90)
    .filter(SizeTable.size.in_(['230','290']))
    .group_by(ProductInfoTable.sku)
    .order_by(ProductInfoTable.sku.desc())
    .limit(5)
    .all()
)

# Print the result
for row in result:
    print(row.sku) 

89
87
86
84
83


In [101]:
session.query(func.group_concat(SizeTable.size).label('concatenated_values')).filter(SizeTable.sku.in_([1,2,3])).group_by(SizeTable.sku).all()

[('230,235,240,245,250',),
 ('230,235,240,245,250,255,260',),
 ('230,235,280,285,290',)]

In [110]:
from sqlalchemy import func

# Assuming session and ProductInfoTable, SizeTable are already defined

# First Query
result = (
    session.query(ProductInfoTable)
    .join(SizeTable, ProductInfoTable.sku == SizeTable.sku)
    .filter(ProductInfoTable.sku < 90)
    .filter(SizeTable.size.in_(['230', '290']))
    .group_by(ProductInfoTable.sku)
    .order_by(ProductInfoTable.sku.desc())
    .limit(5)
    .all()
)

# Extract SKUs from the first result
sku_list = [row.sku for row in result]

# Second Query
second_result = (
    session.query(SizeTable.sku, func.group_concat(SizeTable.size).label('concatenated_values'))
    .filter(SizeTable.sku.in_(sku_list))
    .group_by(SizeTable.sku)
    .all()
)

# Combine esults in Python
final_result = [(row[0], row[1]) for row in second_result]

print(final_result)


[(83, '230,235,260,250,225,245,240,285,280,295,270,255'), (84, '255,270,280,265,230,240,285,295,235,220'), (86, '275,235,240,270,295,230,255,280,250,225'), (87, '230,265,220,255,235,300'), (89, '285,235,275,255,295,290,245,280,240,260')]


In [121]:
# brand category 쉬움
# price 쉬움
# 사이즈 근본적으로 join이 들어가야함
# 사이즈가 있고 없고에 따라서 또 조건이 생김

# 1. 사이즈가 필터가 있는 경우 => join 으로
# 2. 사이즈 필터가 없는 경우 => join 없이 


x = {1:2,3:4}


2 4
